In [76]:
# import the necessary packages
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)


In [77]:
#loading the data as traindata and test data 
#The compose function allows for multiple transforms
#transforms.ToTensor() converts our PILImage to a tensor of shape (C x H x W) in the range [0,1]
#transforms.Normalize(mean,std) normalizes a tensor to a (mean, std) for (R, G, B)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.ImageFolder('C:/Users/Mayank/Downloads/trainimages',transform=transform)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=4,shuffle=True, num_workers=2)
test_set = torchvision.datasets.ImageFolder('C:/Users/Mayank/Downloads/testimagess',transform=transform)
testloader = torch.utils.data.DataLoader(test_set, batch_size=4,shuffle=True, num_workers=2)

In [78]:
#different classes for classification
classes=('CEREAL','CHOCOLATE','COFFEE','HONEY','JAM','JUICE','NUTS','SPICES','TOMATO_SAUCE','MILK')

In [79]:
from torch.utils.data.sampler import SubsetRandomSampler

#Training
n_training_samples = 2000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
n_val_samples = 100
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 800
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

In [80]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1) # convolutional layer
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # convolutional layer
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3,padding=1)
        self.pool = nn.MaxPool2d(2, 2,0) # max pooling layer
        #self.drop2d = nn.Dropout2d(0.2)
        self.fc1 = nn.Linear(128*8*8, 64) # densely connected layer
        self.fc2 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 128*8*8)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [81]:
def outputSize(in_size, kernel_size, stride, padding):

    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1

    return(output)

In [82]:
#DataLoader takes in a dataset and a sampler for loading (num_workers deals with system level memory) 
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=2)
    return(train_loader)

In [83]:
#Test loader have constant batch sizes, so we can define them directly
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)

In [84]:
#loss and optimizer function
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [85]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data[0]
            total_train_loss += loss_size.data[0]
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data[0]
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [86]:
CNN = SimpleCNN()
trainNet(CNN, batch_size=64, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 64
epochs= 5
learning_rate= 0.001


C:\Users\Mayank\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
C:\Users\Mayank\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 1, 12% 	 train_loss: 3.04 took: 20.29s
Epoch 1, 25% 	 train_loss: 2.99 took: 17.05s
Epoch 1, 37% 	 train_loss: 2.97 took: 17.02s
Epoch 1, 50% 	 train_loss: 2.99 took: 17.13s
Epoch 1, 62% 	 train_loss: 2.98 took: 17.13s
Epoch 1, 75% 	 train_loss: 2.93 took: 16.82s
Epoch 1, 87% 	 train_loss: 2.91 took: 17.11s
Epoch 1, 100% 	 train_loss: 2.95 took: 12.98s


C:\Users\Mayank\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Validation loss = 3.28
Epoch 2, 12% 	 train_loss: 2.78 took: 24.61s
Epoch 2, 25% 	 train_loss: 2.76 took: 23.27s
Epoch 2, 37% 	 train_loss: 2.75 took: 18.54s
Epoch 2, 50% 	 train_loss: 2.73 took: 17.65s
Epoch 2, 62% 	 train_loss: 2.67 took: 17.51s
Epoch 2, 75% 	 train_loss: 2.66 took: 17.93s
Epoch 2, 87% 	 train_loss: 2.60 took: 17.84s
Epoch 2, 100% 	 train_loss: 2.60 took: 14.23s
Validation loss = 4.41
Epoch 3, 12% 	 train_loss: 2.60 took: 21.39s
Epoch 3, 25% 	 train_loss: 2.49 took: 17.23s
Epoch 3, 37% 	 train_loss: 2.60 took: 17.54s
Epoch 3, 50% 	 train_loss: 2.58 took: 17.59s
Epoch 3, 62% 	 train_loss: 2.40 took: 16.99s
Epoch 3, 75% 	 train_loss: 2.54 took: 16.93s
Epoch 3, 87% 	 train_loss: 2.34 took: 16.87s
Epoch 3, 100% 	 train_loss: 2.41 took: 13.53s
Validation loss = 4.56
Epoch 4, 12% 	 train_loss: 2.38 took: 21.71s
Epoch 4, 25% 	 train_loss: 2.44 took: 18.27s
Epoch 4, 37% 	 train_loss: 2.33 took: 17.53s
Epoch 4, 50% 	 train_loss: 2.41 took: 17.39s
Epoch 4, 62% 	 train_loss: 2.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

  JAM CEREAL  MILK HONEY


In [72]:
outputss=CNN(images)

In [73]:
_, predicted = torch.max(outputss, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:  JUICE  NUTS CEREAL CHOCOLATE


In [74]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 25 %


In [75]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs =CNN(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of CEREAL : 45 %
Accuracy of CHOCOLATE : 32 %
Accuracy of COFFEE : 20 %
Accuracy of HONEY :  9 %
Accuracy of   JAM : 51 %
Accuracy of JUICE : 67 %
Accuracy of  NUTS : 22 %
Accuracy of SPICES :  5 %
Accuracy of TOMATO_SAUCE :  9 %
Accuracy of PASTA :  0 %
